In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import OneClassSVM

*Считывание и обработка таблиц с данными по месяцам*

In [98]:

df_01_2022 = pd.read_excel('01.2022.xls', index_col=0, comment='#' , header = [1])
df_01_2022['dt'] = '2022-01-01'
df_01_2022['dt'] =  pd.to_datetime(df_01_2022.dt)
df_01_2022 = df_01_2022.reset_index()
df_01_2022 = df_01_2022[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_01_2022 = df_01_2022.set_index(['dt','№ ОДПУ'])
df_01_2022.head(10)


Вид энерг-а ГВС  \
dt         № ОДПУ                      
2022-01-01 126194                NaN   
           147256                NaN   
           148988                NaN   
           1740725               NaN   
           214255                NaN   
           235684                NaN   
           247368                NaN   
           327500207             NaN   
           596                   NaN   
           98735                 NaN   

                                                          Адрес объекта  \
dt         № ОДПУ                                                         
2022-01-01 126194                            г Уфа, пр-кт Октября, д.31   
           147256                          г Уфа, ул. Петергофская, д.2   
           148988                           г Уфа, ул. Российская, д.45   
           1740725                             г Уфа, ул. Кирова, д.103   
           214255                  г Уфа, ул. Российская, д.25 корп.А-Б   
           235684     г Уфа, ул. 8 Марта, д.12 корп.3, Подобъект №49931   
           247368                    г Уфа, ул. Трамвайная, д.13 корп.1   
           327500207                      г Уфа, ул. Карла Маркса, д.40   
           596                                        респ Башкортостан   
           98735                         г Уфа, ул. Достоевского, д.112   

                              Тип объекта Дата текущего показания  \
dt         № ОДПУ                                                   
2022-01-01 126194         Другое строение                     NaT   
           147256         Другое строение              2022-02-01   
           148988     Многоквартирный дом                     NaT   
           1740725    Многоквартирный дом              2022-02-01   
           214255     Многоквартирный дом                     NaT   
           235684     Многоквартирный дом                     NaT   
           247368         Другое строение              2022-02-01   
           327500207  Многоквартирный дом                     NaT   
           596            Другое строение                     NaT   
           98735      Многоквартирный дом              2022-02-01   

                      Текущее потребление, Гкал  
dt         № ОДПУ                                
2022-01-01 126194                           NaN  
           147256                        24.472  
           148988                           NaN  
           1740725                       74.652  
           214255                           NaN  
           235684                           NaN  
           247368                        20.781  
           327500207                        NaN  
           596                              NaN  
           98735                        167.411

In [99]:
# Уберем город из адресов, тк он везде один
df_01_2022['Адрес объекта'] = df_01_2022["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

In [4]:
df_01_2022.head(10)


Вид энерг-а ГВС  \
dt         № ОДПУ                      
2022-01-01 126194                NaN   
           147256                NaN   
           148988                NaN   
           1740725               NaN   
           214255                NaN   
           235684                NaN   
           247368                NaN   
           327500207             NaN   
           596                   NaN   
           98735                 NaN   

                                                    Адрес объекта  \
dt         № ОДПУ                                                   
2022-01-01 126194                             пр-кт Октября, д.31   
           147256                           ул. Петергофская, д.2   
           148988                            ул. Российская, д.45   
           1740725                              ул. Кирова, д.103   
           214255                   ул. Российская, д.25 корп.А-Б   
           235684      ул. 8 Марта, д.12 корп.3, Подобъект №49931   
           247368                     ул. Трамвайная, д.13 корп.1   
           327500207                       ул. Карла Маркса, д.40   
           596                                  респ Башкортостан   
           98735                          ул. Достоевского, д.112   

                              Тип объекта Дата текущего показания  \
dt         № ОДПУ                                                   
2022-01-01 126194         Другое строение                     NaT   
           147256         Другое строение              2022-02-01   
           148988     Многоквартирный дом                     NaT   
           1740725    Многоквартирный дом              2022-02-01   
           214255     Многоквартирный дом                     NaT   
           235684     Многоквартирный дом                     NaT   
           247368         Другое строение              2022-02-01   
           327500207  Многоквартирный дом                     NaT   
           596            Другое строение                     NaT   
           98735      Многоквартирный дом              2022-02-01   

                      Текущее потребление, Гкал  
dt         № ОДПУ                                
2022-01-01 126194                           NaN  
           147256                        24.472  
           148988                           NaN  
           1740725                       74.652  
           214255                           NaN  
           235684                           NaN  
           247368                        20.781  
           327500207                        NaN  
           596                              NaN  
           98735                        167.411

In [5]:
df_01_2022.isna().sum()

Вид энерг-а ГВС              3080
Адрес объекта                   0
Тип объекта                     0
Дата текущего показания       138
Текущее потребление, Гкал     138
dtype: int64

In [6]:
df_01_2022.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4359 entries, (Timestamp('2022-01-01 00:00:00'), '126194') to (Timestamp('2022-01-01 00:00:00'), '260327')
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Вид энерг-а ГВС            1279 non-null   object        
 1   Адрес объекта              4359 non-null   object        
 2   Тип объекта                4359 non-null   object        
 3   Дата текущего показания    4221 non-null   datetime64[ns]
 4   Текущее потребление, Гкал  4221 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 346.3+ KB


In [100]:
df_01_2022 = df_01_2022.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])
#уберем nan, если данных нет в 'Дата текущего показания' и  'Текущее потребление, Гкал'

#df_01_2022['Дата текущего показания'] = df_01_2022['Дата текущего показания'].fillna('2022-02-01')

In [77]:
# убираю 'Дата текущего показания'
#df_01_2022 = df_01_2022[['Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Текущее потребление, Гкал']]
#df_01_2022.head()

Вид энерг-а ГВС                 Адрес объекта  \
dt         № ОДПУ                                                  
2022-01-01 147256              NaN         ул. Петергофская, д.2   
           1740725             NaN             ул. Кирова, д.103   
           247368              NaN   ул. Трамвайная, д.13 корп.1   
           98735               NaN       ул. Достоевского, д.112   
           28884           ГВС-ИТП    ул. Рудольфа Нуреева, д.21   

                            Тип объекта  Текущее потребление, Гкал  
dt         № ОДПУ                                                   
2022-01-01 147256       Другое строение                     24.472  
           1740725  Многоквартирный дом                     74.652  
           247368       Другое строение                     20.781  
           98735    Многоквартирный дом                    167.411  
           28884    Многоквартирный дом                    282.680

In [8]:
df_01_2022[df_01_2022['Вид энерг-а ГВС'] == 'ГВС (централ)']

Вид энерг-а ГВС  \
dt         № ОДПУ                                   
2022-01-01 102922, 105388, 106707   ГВС (централ)   
           176464ГВС                ГВС (централ)   
           280519ГВС                ГВС (централ)   
           101548гвс                ГВС (централ)   
           102575гвс                ГВС (централ)   
           94791 гвс в АБК          ГВС (централ)   
           00133281ГВС              ГВС (централ)   
           231115ГВС                ГВС (централ)   
           1255411/гвс              ГВС (централ)   
           100758гвс                ГВС (централ)   
           100846гвс                ГВС (централ)   

                                                          Адрес объекта  \
dt         № ОДПУ                                                         
2022-01-01 102922, 105388, 106707                 ул. Менделеева, д.207   
           176464ГВС                             ул. Краснодонская, д.3   
           280519ГВС                              б-р. Ибрагимова, д.42   
           101548гвс                                  ул. Свободы, д.29   
           102575гвс                                  ул. Свободы, д.29   
           94791 гвс в АБК                  ул. Кольцевая, д.173 корп.1   
           00133281ГВС                       ул. 50-летия Октября, д.32   
           231115ГВС                             ул. Краснодонская, д.5   
           1255411/гвс               ул. Георгия Мушникова, д.21 корп.а   
           100758гвс                ул. Свободы, д.29, Подобъект №31215   
           100846гвс                                  ул. Свободы, д.29   

                                           Тип объекта  \
dt         № ОДПУ                                        
2022-01-01 102922, 105388, 106707  Многоквартирный дом   
           176464ГВС               Многоквартирный дом   
           280519ГВС               Многоквартирный дом   
           101548гвс                   Другое строение   
           102575гвс                   Другое строение   
           94791 гвс в АБК             Другое строение   
           00133281ГВС                 Другое строение   
           231115ГВС               Многоквартирный дом   
           1255411/гвс                 Другое строение   
           100758гвс                   Другое строение   
           100846гвс                   Другое строение   

                                  Дата текущего показания  \
dt         № ОДПУ                                           
2022-01-01 102922, 105388, 106707              2022-02-01   
           176464ГВС                           2022-02-01   
           280519ГВС                           2022-02-01   
           101548гвс                           2022-02-01   
           102575гвс                           2022-02-01   
           94791 гвс в АБК                     2022-02-01   
           00133281ГВС                         2022-02-01   
           231115ГВС                           2022-02-01   
           1255411/гвс                         2022-02-01   
           100758гвс                           2022-02-01   
           100846гвс                           2022-02-01   

                                   Текущее потребление, Гкал  
dt         № ОДПУ                                             
2022-01-01 102922, 105388, 106707                    381.266  
           176464ГВС                                  66.574  
           280519ГВС                                  44.529  
           101548гвс                                   0.639  
           102575гвс                                   7.306  
           94791 гвс в АБК                             1.233  
           00133281ГВС                                18.807  
           231115ГВС                                  75.562  
           1255411/гвс                                10.370  
           100758гвс                                   2.827  
           100846гвс                                   0.304

In [101]:

# убираю ГВС (централ) по условим задачи
df_01_2022 = df_01_2022[df_01_2022['Вид энерг-а ГВС'] != 'ГВС (централ)']



In [102]:
#проверим, что ГВС (централ) отсутствует
df_01_2022[df_01_2022['Вид энерг-а ГВС'] == 'ГВС (централ)']

,,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания,"Текущее потребление, Гкал"
dt,№ ОДПУ,,,,,


In [11]:
df_01_2022.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4210 entries, (Timestamp('2022-01-01 00:00:00'), '147256') to (Timestamp('2022-01-01 00:00:00'), '260327')
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Вид энерг-а ГВС            1232 non-null   object        
 1   Адрес объекта              4210 non-null   object        
 2   Тип объекта                4210 non-null   object        
 3   Дата текущего показания    4210 non-null   datetime64[ns]
 4   Текущее потребление, Гкал  4210 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 340.0+ KB


In [98]:
df_01_2022.describe()

,Дата текущего показания,"Текущее потребление, Гкал"
count,4210,4210.000000
mean,2022-01-31 23:49:23.800475136,171.578223
min,2022-01-01 00:00:00,0.000000
25%,2022-02-01 00:00:00,86.620000
50%,2022-02-01 00:00:00,120.841500
75%,2022-02-01 00:00:00,181.062750
max,2022-02-01 00:00:00,26768.000000
std,NaN,516.184761


In [103]:
#df_01_2022.sort_value('Текущее потребление, Гкал')
df_01_2022.sort_values(by='Текущее потребление, Гкал')

Вид энерг-а ГВС  \
dt         № ОДПУ                         
2022-01-01 13493                ГВС-ИТП   
           18-028134                NaN   
           20-114624                NaN   
           01629181922              NaN   
           103207                   NaN   
...                                 ...   
           20-106837            ГВС-ИТП   
           21753                ГВС-ИТП   
           CG166, CF944             NaN   
           2300 N CF944             NaN   
           595                  ГВС-ИТП   

                                                         Адрес объекта  \
dt         № ОДПУ                                                        
2022-01-01 13493                            ул. 50-летия Октября, д.24   
           18-028134      ул. Маршала Жукова, д.1/2, Подобъект №984860   
           20-114624                         пр-кт Октября, д.3 корп.1   
           01629181922              ул. Рудольфа Нуреева, д.1 корп.1-3   
           103207                             ул. Вологодская, д.146 А   
...                                                                ...   
           20-106837                        ул. 50-летия Октября, д.34   
           21753                                   проезд. Лесной, д.3   
           CG166, CF944                                 ул. Трамвайная   
           2300 N CF944                            ул. Трамвайная, д.3   
           595                                       респ Башкортостан   

                                 Тип объекта Дата текущего показания  \
dt         № ОДПУ                                                      
2022-01-01 13493             Другое строение              2022-02-01   
           18-028134     Многоквартирный дом              2022-02-01   
           20-114624     Многоквартирный дом              2022-02-01   
           01629181922   Многоквартирный дом              2022-02-01   
           103207            Другое строение              2022-02-01   
...                                      ...                     ...   
           20-106837         Другое строение              2022-02-01   
           21753             Другое строение              2022-02-01   
           CG166, CF944      Другое строение              2022-02-01   
           2300 N CF944      Другое строение              2022-02-01   
           595               Другое строение              2022-02-01   

                         Текущее потребление, Гкал  
dt         № ОДПУ                                   
2022-01-01 13493                            0.0000  
           18-028134                        0.0000  
           20-114624                        0.6640  
           01629181922                      0.7290  
           103207                           0.7700  
...                                            ...  
           20-106837                     2238.9950  
           21753                         2660.4185  
           CG166, CF944                 10451.1400  
           2300 N CF944                 14256.1500  
           595                          26768.0000  

[4210 rows x 5 columns]


*Обработаем оставшиеся таблицы и запишем результат в отдельный csv файл*


In [104]:
df_01_2023 = pd.read_excel('01.2023.xlsx', index_col=0, comment='#' , header = [1])
df_01_2023['dt'] = '2023-01-01'
df_01_2023['dt'] =  pd.to_datetime(df_01_2023.dt)
df_01_2023 = df_01_2023.reset_index()
df_01_2023 = df_01_2023[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_01_2023 = df_01_2023.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_01_2023['Адрес объекта'] = df_01_2023["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

df_01_2023.head()

Вид энерг-а ГВС          Адрес объекта  \
dt         № ОДПУ                                            
2023-01-01 787              ГВС-ИТП                  г Уфа   
           00119540         ГВС-ИТП   б-р. Баландина, д.11   
           30822            ГВС-ИТП    б-р. Баландина, д.2   
           237959               NaN   б-р. Баландина, д.2а   
           145046           ГВС-ИТП    б-р. Баландина, д.4   

                             Тип объекта Дата текущего показания  \
dt         № ОДПУ                                                  
2023-01-01 787           Другое строение              2023-02-01   
           00119540  Многоквартирный дом              2023-02-01   
           30822     Многоквартирный дом              2023-02-01   
           237959        Другое строение              2023-02-01   
           145046    Многоквартирный дом              2023-02-01   

                     Текущее потребление, Гкал  
dt         № ОДПУ                               
2023-01-01 787                         188.000  
           00119540                    261.323  
           30822                       459.310  
           237959                       32.872  
           145046                      168.030

In [16]:
df_01_2023.isna().sum()

Вид энерг-а ГВС              3047
Адрес объекта                   0
Тип объекта                     0
Дата текущего показания         0
Текущее потребление, Гкал       0
dtype: int64

In [105]:
df_01_2023 = df_01_2023.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_01_2023 = df_01_2023[df_01_2023['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_01_2023.head()

Вид энерг-а ГВС          Адрес объекта  \
dt         № ОДПУ                                            
2023-01-01 787              ГВС-ИТП                  г Уфа   
           00119540         ГВС-ИТП   б-р. Баландина, д.11   
           30822            ГВС-ИТП    б-р. Баландина, д.2   
           237959               NaN   б-р. Баландина, д.2а   
           145046           ГВС-ИТП    б-р. Баландина, д.4   

                             Тип объекта Дата текущего показания  \
dt         № ОДПУ                                                  
2023-01-01 787           Другое строение              2023-02-01   
           00119540  Многоквартирный дом              2023-02-01   
           30822     Многоквартирный дом              2023-02-01   
           237959        Другое строение              2023-02-01   
           145046    Многоквартирный дом              2023-02-01   

                     Текущее потребление, Гкал  
dt         № ОДПУ                               
2023-01-01 787                         188.000  
           00119540                    261.323  
           30822                       459.310  
           237959                       32.872  
           145046                      168.030

In [106]:
#len(df_01_2023)
df_all = pd.concat([df_01_2022,df_01_2023])
len(df_all)



8536

In [107]:
df_02_2022 = pd.read_excel('02.2022.xls', index_col=0, comment='#' , header = [1])
df_02_2022['dt'] = '2022-02-01'
df_02_2022['dt'] =  pd.to_datetime(df_02_2022.dt)
df_02_2022 = df_02_2022.reset_index()
df_02_2022 = df_02_2022[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_02_2022 = df_02_2022.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_02_2022['Адрес объекта'] = df_02_2022["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()

In [108]:
df_02_2022 = df_02_2022.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_02_2022 = df_02_2022[df_02_2022['Вид энерг-а ГВС'] != 'ГВС (централ)']
#df_02_2022.head()

len(df_02_2022)

4219

In [109]:
df_all = pd.concat([df_all, df_02_2022])


In [110]:
len(df_all)
#12755

12755

In [111]:
df_02_2023 = pd.read_excel('02.2023.xlsx', index_col=0, comment='#' , header = [1])
df_02_2023['dt'] = '2023-02-01'
df_02_2023['dt'] =  pd.to_datetime(df_02_2023.dt)
df_02_2023 = df_02_2023.reset_index()
df_02_2023 = df_02_2023[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_02_2023 = df_02_2023.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_02_2023['Адрес объекта'] = df_02_2023["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_02_2023 = df_02_2023.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_02_2023 = df_02_2023[df_02_2023['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_02_2023.head()

Вид энерг-а ГВС          Адрес объекта  \
dt         № ОДПУ                                            
2023-02-01 787              ГВС-ИТП                  г Уфа   
           00119540         ГВС-ИТП   б-р. Баландина, д.11   
           30822            ГВС-ИТП    б-р. Баландина, д.2   
           237959               NaN   б-р. Баландина, д.2а   
           145046           ГВС-ИТП    б-р. Баландина, д.4   

                             Тип объекта Дата текущего показания  \
dt         № ОДПУ                                                  
2023-02-01 787           Другое строение              2023-03-01   
           00119540  Многоквартирный дом              2023-03-01   
           30822     Многоквартирный дом              2023-03-01   
           237959        Другое строение              2023-03-01   
           145046    Многоквартирный дом              2023-03-01   

                     Текущее потребление, Гкал  
dt         № ОДПУ                               
2023-02-01 787                         148.000  
           00119540                    210.563  
           30822                       359.570  
           237959                       33.400  
           145046                      130.670

In [113]:
df_all = pd.concat([df_all, df_02_2023])
len(df_all)
#17090

17090

In [114]:
df_03_2022 = pd.read_excel('03.2022.xls', index_col=0, comment='#' , header = [1])
df_03_2022['dt'] = '2022-03-01'
df_03_2022['dt'] =  pd.to_datetime(df_03_2022.dt)
df_03_2022 = df_03_2022.reset_index()
df_03_2022 = df_03_2022[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_03_2022 = df_03_2022.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_03_2022['Адрес объекта'] = df_03_2022["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_03_2022 = df_03_2022.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_03_2022 = df_03_2022[df_03_2022['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_03_2022.head()
len(df_03_2022)

4213

In [116]:
#df_all = pd.concat([df_all, df_03_2022])
len(df_all)
#21303

21303

In [117]:
df_03_2023 = pd.read_excel('03.2023.xlsx', index_col=0, comment='#' , header = [1])
df_03_2023['dt'] = '2023-03-01'
df_03_2023['dt'] =  pd.to_datetime(df_03_2023.dt)
df_03_2023 = df_03_2023.reset_index()
df_03_2023 = df_03_2023[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_03_2023 = df_03_2023.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_03_2023['Адрес объекта'] = df_03_2023["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_03_2023 = df_03_2023.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_03_2023 = df_03_2023[df_03_2023['Вид энерг-а ГВС'] != 'ГВС (централ)']
#df_03_2023.head()
len(df_03_2023)

4343

In [119]:
#df_all = pd.concat([df_all, df_03_2023])
len(df_all)
#25646

25646

In [120]:
df_04_2022 = pd.read_excel('04.2022.xls', index_col=0, comment='#' , header = [1])
df_04_2022['dt'] = '2022-04-01'
df_04_2022['dt'] =  pd.to_datetime(df_04_2022.dt)
df_04_2022 = df_04_2022.reset_index()
df_04_2022 = df_04_2022[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_04_2022 = df_04_2022.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_04_2022['Адрес объекта'] = df_04_2022["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_04_2022 = df_04_2022.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_04_2022 = df_04_2022[df_04_2022['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_04_2022.head()
len(df_04_2022)

4194

In [122]:
#df_all = pd.concat([df_all, df_04_2022])
len(df_all)
#29840

29840

In [123]:
df_04_2023 = pd.read_excel('04.2023.xlsx', index_col=0, comment='#' , header = [1])
df_04_2023['dt'] = '2023-04-01'
df_04_2023['dt'] =  pd.to_datetime(df_04_2023.dt)
df_04_2023 = df_04_2023.reset_index()
df_04_2023 = df_04_2023[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_04_2023 = df_04_2023.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_04_2023['Адрес объекта'] = df_04_2023["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_04_2023 = df_04_2023.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_04_2023 = df_04_2023[df_04_2023['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_04_2023.head()
len(df_04_2023)

4282

In [126]:
#df_all = pd.concat([df_all, df_04_2023])
len(df_all)
#34122

34122

In [57]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 34122 entries, (Timestamp('2022-01-01 00:00:00'), '147256') to (Timestamp('2023-04-01 00:00:00'), '1981')
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Вид энерг-а ГВС            10099 non-null  object        
 1   Адрес объекта              34122 non-null  object        
 2   Тип объекта                34122 non-null  object        
 3   Дата текущего показания    34122 non-null  datetime64[ns]
 4   Текущее потребление, Гкал  34122 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 1.6+ MB


In [127]:
df_05_2022 = pd.read_excel('05.2022.xls', index_col=0, comment='#' , header = [1])
df_05_2022['dt'] = '2022-05-01'
df_05_2022['dt'] =  pd.to_datetime(df_05_2022.dt)
df_05_2022 = df_05_2022.reset_index()
df_05_2022 = df_05_2022[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_05_2022 = df_05_2022.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_05_2022['Адрес объекта'] = df_05_2022["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_05_2022 = df_05_2022.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_05_2022 = df_05_2022[df_05_2022['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_05_2022.head()
len(df_05_2022)

1174

In [130]:
#df_all = pd.concat([df_all, df_05_2022])
len(df_all)
#35296

35296

In [131]:
df_05_2023 = pd.read_excel('05.2023.xlsx', index_col=0, comment='#' , header = [1])
df_05_2023['dt'] = '2023-05-01'
df_05_2023['dt'] =  pd.to_datetime(df_05_2023.dt)
df_05_2023 = df_05_2023.reset_index()
df_05_2023 = df_05_2023[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_05_2023 = df_05_2023.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_05_2023['Адрес объекта'] = df_05_2023["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_05_2023 = df_05_2023.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_05_2023 = df_05_2023[df_05_2023['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_05_2023.head()
len(df_05_2023)

1160

In [134]:
#df_all = pd.concat([df_all, df_05_2023])
len(df_all)
#36456

36456

In [135]:
df_06_2022 = pd.read_excel('06.2022.xls', index_col=0, comment='#' , header = [1])
df_06_2022['dt'] = '2022-06-01'
df_06_2022['dt'] =  pd.to_datetime(df_06_2022.dt)
df_06_2022 = df_06_2022.reset_index()
df_06_2022 = df_06_2022[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_06_2022 = df_06_2022.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_06_2022['Адрес объекта'] = df_06_2022["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_06_2022 = df_06_2022.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_06_2022 = df_06_2022[df_06_2022['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_06_2022.head()
len(df_06_2022)

1066

In [138]:
#df_all = pd.concat([df_all, df_06_2022])
len(df_all)
#37522

37522

In [139]:
df_06_2023 = pd.read_excel('06.2023.xlsx', index_col=0, comment='#' , header = [1])
df_06_2023['dt'] = '2023-06-01'
df_06_2023['dt'] =  pd.to_datetime(df_06_2023.dt)
df_06_2023 = df_06_2023.reset_index()
df_06_2023 = df_06_2023[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_06_2023 = df_06_2023.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_06_2023['Адрес объекта'] = df_06_2023["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_06_2023 = df_06_2023.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_06_2023 = df_06_2023[df_06_2023['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_06_2023.head()
len(df_06_2023)

1106

In [142]:
#df_all = pd.concat([df_all, df_06_2023])
len(df_all)
#38628


38628

In [143]:
df_07_2021 = pd.read_excel('07.2021.xls', index_col=0, comment='#' , header = [1])
df_07_2021['dt'] = '2021-07-01'
df_07_2021['dt'] =  pd.to_datetime(df_07_2021.dt)
df_07_2021 = df_07_2021.reset_index()
df_07_2021 = df_07_2021[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_07_2021 = df_07_2021.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_07_2021['Адрес объекта'] = df_07_2021["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_07_2021 = df_07_2021.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_07_2021 = df_07_2021[df_07_2021['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_07_2021.head()
len(df_07_2021)


986

In [146]:
#df_all = pd.concat([df_all, df_07_2021])
len(df_all)
#39614



39614

In [147]:
df_07_2022 = pd.read_excel('07.2022.xlsx', index_col=0, comment='#' , header = [1])
df_07_2022['dt'] = '2022-07-01'
df_07_2022['dt'] =  pd.to_datetime(df_07_2022.dt)
df_07_2022 = df_07_2022.reset_index()
df_07_2022 = df_07_2022[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_07_2022 = df_07_2022.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_07_2022['Адрес объекта'] = df_07_2022["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_07_2022 = df_07_2022.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_07_2022 = df_07_2022[df_07_2022['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_07_2022.head()
len(df_07_2022)

1050

In [150]:
#df_all = pd.concat([df_all, df_07_2022])
len(df_all)
#40664
#df_all.info()

40664

In [151]:
df_08_2021 = pd.read_excel('08.2021.xls', index_col=0, comment='#' , header = [1])
df_08_2021['dt'] = '2021-08-01'
df_08_2021['dt'] =  pd.to_datetime(df_08_2021.dt)
df_08_2021 = df_08_2021.reset_index()
df_08_2021 = df_08_2021[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_08_2021 = df_08_2021.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_08_2021['Адрес объекта'] = df_08_2021["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_08_2021 = df_08_2021.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_08_2021 = df_08_2021[df_08_2021['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_08_2021.head()
len(df_08_2021)

974

In [154]:
#df_all = pd.concat([df_all, df_08_2021])
len(df_all)
#41638
#df_all.info()

41638

In [155]:
df_08_2022 = pd.read_excel('08.2022.xlsx', index_col=0, comment='#' , header = [1])
df_08_2022['dt'] = '2022-08-01'
df_08_2022['dt'] =  pd.to_datetime(df_08_2022.dt)
df_08_2022 = df_08_2022.reset_index()
df_08_2022 = df_08_2022[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_08_2022 = df_08_2022.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_08_2022['Адрес объекта'] = df_08_2022["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_08_2022 = df_08_2022.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_08_2022 = df_08_2022[df_08_2022['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_08_2022.head()
len(df_08_2022)

1020

In [158]:
#df_all = pd.concat([df_all, df_08_2022])
len(df_all)
#42658
#df_all.info()

42658

In [159]:
df_09_2021 = pd.read_excel('09.2021.xls', index_col=0, comment='#' , header = [1])
df_09_2021['dt'] = '2021-09-01'
df_09_2021['dt'] =  pd.to_datetime(df_09_2021.dt)
df_09_2021 = df_09_2021.reset_index()
df_09_2021 = df_09_2021[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_09_2021 = df_09_2021.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_09_2021['Адрес объекта'] = df_09_2021["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_09_2021 = df_09_2021.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_09_2021 = df_09_2021[df_09_2021['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_09_2021.head()
len(df_09_2021)

1482

In [164]:
#df_all = pd.concat([df_all, df_09_2021])
len(df_all)
#44140
#df_all.info()

44140

In [165]:
df_09_2022 = pd.read_excel('09.2022.xlsx', index_col=0, comment='#' , header = [1])
df_09_2022['dt'] = '2022-09-01'
df_09_2022['dt'] =  pd.to_datetime(df_09_2022.dt)
df_09_2022 = df_09_2022.reset_index()
df_09_2022 = df_09_2022[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_09_2022 = df_09_2022.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_09_2022['Адрес объекта'] = df_09_2022["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_09_2022 = df_09_2022.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_09_2022 = df_09_2022[df_09_2022['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_09_2022.head()
len(df_09_2022)

1080

In [169]:
#df_all = pd.concat([df_all, df_09_2022])
len(df_all)
#45220
#df_all.info()

45220

In [170]:
df_10_2021 = pd.read_excel('10.2021.xls', index_col=0, comment='#' , header = [1])
df_10_2021['dt'] = '2021-10-01'
df_10_2021['dt'] =  pd.to_datetime(df_10_2021.dt)
df_10_2021 = df_10_2021.reset_index()
df_10_2021 = df_10_2021[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_10_2021 = df_10_2021.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_10_2021['Адрес объекта'] = df_10_2021["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_10_2021 = df_10_2021.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_10_2021 = df_10_2021[df_10_2021['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_10_2021.head()
len(df_10_2021)

4037

In [173]:
#df_all = pd.concat([df_all, df_10_2021])
len(df_all)
#49257
#df_all.info()

49257

In [174]:
df_10_2022 = pd.read_excel('10.2022.xlsx', index_col=0, comment='#' , header = [1])
df_10_2022['dt'] = '2022-10-01'
df_10_2022['dt'] =  pd.to_datetime(df_10_2022.dt)
df_10_2022 = df_10_2022.reset_index()
df_10_2022 = df_10_2022[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_10_2022 = df_10_2022.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_10_2022['Адрес объекта'] = df_10_2022["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_10_2022 = df_10_2022.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_10_2022 = df_10_2022[df_10_2022['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_10_2022.head()
len(df_10_2022)

4095

In [177]:
#df_all = pd.concat([df_all, df_10_2022])
len(df_all)
#53352
#df_all.info()

53352

In [178]:
df_11_2021 = pd.read_excel('11.2021.xls', index_col=0, comment='#' , header = [1])
df_11_2021['dt'] = '2021-11-01'
df_11_2021['dt'] =  pd.to_datetime(df_11_2021.dt)
df_11_2021 = df_11_2021.reset_index()
df_11_2021 = df_11_2021[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_11_2021 = df_11_2021.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_11_2021['Адрес объекта'] = df_11_2021["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_11_2021 = df_11_2021.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_11_2021 = df_11_2021[df_11_2021['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_11_2021.head()
len(df_11_2021)

4197

In [181]:
#df_all = pd.concat([df_all, df_11_2021])
len(df_all)
#57549
#df_all.info()

57549

In [182]:
df_11_2022 = pd.read_excel('11.2022.xlsx', index_col=0, comment='#' , header = [1])
df_11_2022['dt'] = '2022-11-01'
df_11_2022['dt'] =  pd.to_datetime(df_11_2022.dt)
df_11_2022 = df_11_2022.reset_index()
df_11_2022 = df_11_2022[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_11_2022 = df_11_2022.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_11_2022['Адрес объекта'] = df_11_2022["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_11_2022 = df_11_2022.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_11_2022 = df_11_2022[df_11_2022['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_11_2022.head()
len(df_11_2022)

4294

In [185]:
#df_all = pd.concat([df_all, df_11_2022])
len(df_all)
#61843
#df_all.info()

61843

In [186]:
df_12_2021 = pd.read_excel('12.2021.xls', index_col=0, comment='#' , header = [1])
df_12_2021['dt'] = '2021-12-01'
df_12_2021['dt'] =  pd.to_datetime(df_12_2021.dt)
df_12_2021 = df_12_2021.reset_index()
df_12_2021 = df_12_2021[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_12_2021 = df_12_2021.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_12_2021['Адрес объекта'] = df_12_2021["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_12_2021 = df_12_2021.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_12_2021 = df_12_2021[df_12_2021['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_12_2021.head()
len(df_12_2021)

4234

In [189]:
#df_all = pd.concat([df_all, df_12_2021])
len(df_all)
#66077
#df_all.info()

66077

In [191]:
df_12_2022 = pd.read_excel('12.2022.xlsx', index_col=0, comment='#' , header = [1])
df_12_2022['dt'] = '2022-12-01'
df_12_2022['dt'] =  pd.to_datetime(df_12_2022.dt)
df_12_2022 = df_12_2022.reset_index()
df_12_2022 = df_12_2022[['dt','№ ОДПУ', 'Вид энерг-а ГВС', 'Адрес объекта', 'Тип объекта', 'Дата текущего показания', 'Текущее потребление, Гкал']]
df_12_2022 = df_12_2022.set_index(['dt','№ ОДПУ'])

# Уберем город из адресов
df_12_2022['Адрес объекта'] = df_12_2022["Адрес объекта"].str.replace("г Уфа,", "", regex = True)

#df_02_2022.head()
df_12_2022 = df_12_2022.dropna(subset=['Дата текущего показания', 'Текущее потребление, Гкал'])

# убираю ГВС (централ)
df_12_2022 = df_12_2022[df_12_2022['Вид энерг-а ГВС'] != 'ГВС (централ)']
df_12_2022.head()
len(df_12_2022)

4332

In [195]:
#df_all = pd.concat([df_all, df_12_2022])
len(df_all)
#66077
#df_all.info()

70409

*Получили суммарно 70409 строк после объединения таблиц*

In [197]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 70409 entries, (Timestamp('2022-01-01 00:00:00'), '147256') to (Timestamp('2022-12-01 00:00:00'), '1981')
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Вид энерг-а ГВС            27894 non-null  object        
 1   Адрес объекта              70409 non-null  object        
 2   Тип объекта                70409 non-null  object        
 3   Дата текущего показания    70409 non-null  datetime64[ns]
 4   Текущее потребление, Гкал  70409 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 3.0+ MB


In [199]:
df_all.head()

Вид энерг-а ГВС                 Адрес объекта  \
dt         № ОДПУ                                                  
2022-01-01 147256              NaN         ул. Петергофская, д.2   
           1740725             NaN             ул. Кирова, д.103   
           247368              NaN   ул. Трамвайная, д.13 корп.1   
           98735               NaN       ул. Достоевского, д.112   
           28884           ГВС-ИТП    ул. Рудольфа Нуреева, д.21   

                            Тип объекта Дата текущего показания  \
dt         № ОДПУ                                                 
2022-01-01 147256       Другое строение              2022-02-01   
           1740725  Многоквартирный дом              2022-02-01   
           247368       Другое строение              2022-02-01   
           98735    Многоквартирный дом              2022-02-01   
           28884    Многоквартирный дом              2022-02-01   

                    Текущее потребление, Гкал  
dt         № ОДПУ                              
2022-01-01 147256                      24.472  
           1740725                     74.652  
           247368                      20.781  
           98735                      167.411  
           28884                      282.680

In [201]:
#Записали данные в файл df_all.csv
df_all.to_csv('df_all.csv')

In [304]:
len(df_all['Адрес объекта'].unique())

4243

In [209]:
temp = pd.read_excel('temp.xls', index_col=1, comment='#' , header = [1])
temp.T

Период,"Тн.в, град.С","Продолжительность ОЗП, сут."
Unnamed: 0,NaN,NaN
2021-07-01 00:00:00,NaN,NaN
2021-08-01 00:00:00,NaN,NaN
2021-09-01 00:00:00,NaN,NaN
2021-10-01 00:00:00,6.696552,29.0
2021-11-01 00:00:00,-1.000000,31.0
2021-12-01 00:00:00,-5.176667,30.0
2022-01-01 00:00:00,-11.500000,31.0
2022-02-01 00:00:00,-6.735714,28.0
2022-03-01 00:00:00,-5.151613,31.0


*Добавим столбцы со средней температурой для dt и кол-во дней в месяце*

In [2]:
t = pd.read_html('http://www.pogodaiklimat.ru/history/28722.htm')
temps = t[1].iloc[169:173] #169-2021,170-2022,171-2023,172-2024
temps = temps[[0,1,2,3,4,5,6,7,8,9,10,11]]
temps = temps.T
temps['dt'] = 1,2,3,4,5,6,7,8,9,10,11,12
temps_2021 = temps[[169]]
temps_2022 = temps[[170]]
temps_2023 = temps[[171]]
temps_2021['dt'] = '2021.01.01','2021.02.01','2021.03.01','2021.04.01','2021.05.01','2021.06.01','2021.07.01','2021.08.01','2021.09.01','2021.10.01','2021.11.01','2021.12.01'
temps_2021['dt'] =  pd.to_datetime(temps_2021.dt)

temps_2021.rename(columns = {169:'avg_temp'}, inplace = True)
temps_2021 = temps_2021.set_index('dt')
temps_2021

,avg_temp
dt,
2021-01-01,-10.3
2021-02-01,-14.5
2021-03-01,-6.0
2021-04-01,7.4
2021-05-01,18.5
2021-06-01,20.7
2021-07-01,20.7
2021-08-01,21.4
2021-09-01,10.0


In [3]:
temps_2022['dt'] = '2022.01.01','2022.02.01','2022.03.01','2022.04.01','2022.05.01','2022.06.01','2022.07.01','2022.08.01','2022.09.01','2022.10.01','2022.11.01','2022.12.01'
temps_2022['dt'] =  pd.to_datetime(temps_2022.dt)

temps_2022.rename(columns = {170:'avg_temp'}, inplace = True)
temps_2022 = temps_2022.set_index('dt')
temps_2022

,avg_temp
dt,
2022-01-01,-11.9
2022-02-01,-4.4
2022-03-01,-6.1
2022-04-01,7.8
2022-05-01,10.9
2022-06-01,16.4
2022-07-01,20.4
2022-08-01,19.7
2022-09-01,11.9


In [4]:

temps_2023['dt'] = '2023.01.01','2023.02.01','2023.03.01','2023.04.01','2023.05.01','2023.06.01','2023.07.01','2023.08.01','2023.09.01','2023.10.01','2023.11.01','2023.12.01'
temps_2023['dt'] =  pd.to_datetime(temps_2023.dt)

temps_2023.rename(columns = {171:'avg_temp'}, inplace = True)
temps_2023 = temps_2023.set_index('dt')
# обрезаем до 06.23
temps_2023 = temps_2023.loc['2023-01-01':'2023-06-01']


In [5]:
temps_2023

,avg_temp
dt,
2023-01-01,-14.9
2023-02-01,-9.6
2023-03-01,0.4
2023-04-01,8.7
2023-05-01,15.8
2023-06-01,16.5


In [6]:
# обрезаем 2021
temps_2021 = temps_2021.loc['2021-07-01':'2021-12-01']
temps_2021

,avg_temp
dt,
2021-07-01,20.7
2021-08-01,21.4
2021-09-01,10.0
2021-10-01,5.2
2021-11-01,-2.3
2021-12-01,-8.1


In [7]:
temps_2021_2023 = pd.concat([temps_2021,temps_2022,temps_2023])
temps_2021_2023['days'] = 31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30
temps_2021_2023

,avg_temp,days
dt,,
2021-07-01,20.7,31
2021-08-01,21.4,31
2021-09-01,10.0,30
2021-10-01,5.2,31
2021-11-01,-2.3,30
2021-12-01,-8.1,31
2022-01-01,-11.9,31
2022-02-01,-4.4,28
2022-03-01,-6.1,31


In [40]:


#df_all = pd.read_csv('df_all.csv')


df_all = df_all.reset_index() 
#df_all['dt'] =  pd.to_datetime(df_all.dt)
#df_all = df_all.set_index(['dt'])

In [36]:
#Добавили в таблицу df_all dataFrame c данными по помесячной температуре и с числом дней
df_all = df_all.merge(temps_2021_2023, how = "left", on = "dt")


In [46]:
#df_all = df_all.set_index(['dt','№ ОДПУ'])
#df_all.to_csv('df_all_temp.csv')

#df_all.info()


In [747]:
#Запишем данные в файл df_all_temp.csv
df_all_temp = pd.read_csv('df_all_temp.csv')

In [748]:
#df_all_temp = df_all_temp.set_index(['dt','№ ОДПУ'])
df_all_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70409 entries, 0 to 70408
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   dt                         70409 non-null  object 
 1   № ОДПУ                     70409 non-null  object 
 2   Вид энерг-а ГВС            27894 non-null  object 
 3   Адрес объекта              70409 non-null  object 
 4   Тип объекта                70409 non-null  object 
 5   Дата текущего показания    70409 non-null  object 
 6   Текущее потребление, Гкал  70409 non-null  float64
 7   avg_temp                   70409 non-null  float64
 8   days                       70409 non-null  int64  
dtypes: float64(2), int64(1), object(6)
memory usage: 4.8+ MB


*работа с таблицей Тип строения, этажность, площадь, год постройки.xlsx*



Обработаем таблицу с параметрами зданий
1) уберем город из адресов
2) уберем дубликаты
3) дата постройки - разбить на диапазоны и ранжировать
4) этажи ранжировать

In [834]:
#building_param = pd.read_excel('building_param.xlsx', index_col=0, comment='#' , header = [1])

building_param = building_param.reset_index()


In [835]:
# Уберем город из адресов
building_param['Адрес объекта'] = building_param["Адрес объекта"].str.replace("г Уфа,", "", regex = True)
building_param.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5873 entries, 0 to 5872
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Адрес объекта          5873 non-null   object 
 1   Тип Объекта            5873 non-null   object 
 2   Этажность объекта      5873 non-null   object 
 3   Дата постройки         4403 non-null   object 
 4   Общая площадь объекта  5749 non-null   float64
dtypes: float64(1), object(4)
memory usage: 229.5+ KB


In [836]:

len(building_param['Адрес объекта'].unique())


5695

In [137]:
len(building_param['Адрес объекта'])
#УДАЛИТЬ НЕУНИКАЛЬНЫЕ

5873

In [72]:
#building_param.isna().sum()

Адрес объекта               0
Тип Объекта                 0
Этажность объекта           0
Дата постройки           1470
Общая площадь объекта     124
dtype: int64

In [662]:
building_param_dup = building_param[building_param['Адрес объекта'].duplicated()]
building_param_dup

,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта


In [173]:
#building_param_dup[building_param_dup['Общая площадь объекта'] == 3698.00]['Адрес объекта'].values
#building_param_dup[building_param_dup['Адрес объекта'] == 'ул. Юрия Гагарина, д.64 корп.1']

#building_param_dup['Адрес объекта'] = building_param_dup['Адрес объекта'].str.strip()
#building_param_dup
#df[df['Age'] == value]
#building_param_dup['Адрес объекта']

удаляю дубликаты из building_param
всего адресов - 5873
уникальные адреса - 5695

In [663]:
len(building_param)

5693

In [138]:
#уберем пробелы в адресах
#building_param['Адрес объекта'] = building_param['Адрес объекта'].str.strip()

In [660]:
building_param[building_param['Адрес объекта'] == 'ул. Карла Маркса, д.51 корп.2']

,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
2311,"ул. Карла Маркса, д.51 корп.2",Другое строение,0,NaN,0.01
2312,"ул. Карла Маркса, д.51 корп.2",Многоквартирный дом,16,2003-01-01 00:00:00,6510.52


In [661]:
building_param = building_param.drop(2311)

In [190]:
#building_param[building_param['Адрес объекта'] == 'ул. Чукотская, д.23']['Общая площадь объекта']

5561    22.2
Name: Общая площадь объекта, dtype: float64

In [680]:

#building_param.to_csv('building_param.csv')
#building_param['Дата постройки'] = pd.to_datetime(building_param['Дата постройки']) 



date_str = "2003-01-01 00:00:00"
timestamp = pd.Timestamp(date_str)

# Extract the year
year = timestamp.year

# Convert the year to a string
result = str(year)

print(result) 

2003


In [700]:
#building_param['Дата постройки'] = pd.Timestamp(building_param['Дата постройки']) 
#building_param['Дата постройки'] = pd.to_datetime(building_param['Дата постройки']) 
#building_param['Дата постройки'] = building_param['Дата постройки'].dt.year
building_param.min()
#building_param['Дата постройки'] = building_param['Дата постройки'].astype('Int64')


Адрес объекта            б-р. Баландина, д.11
Тип Объекта                         Автомойка
Этажность объекта                           0
Дата постройки                           1905
Общая площадь объекта                     0.0
dtype: object

In [829]:
#labels = ['NaT', '1900-1958', '1959-1989', '1990-2000', '2001-2010', '2011-2024'] 
len(building_param['Адрес объекта'].unique())


5693

In [709]:
#разбиваем на диапазоны
bins = [1900, 1958, 1989, 2000, 2010, 2024]
labels = ['1900-1958', '1959-1989', '1990-2000', '2001-2010', '2011-2024']

# Разбивка годов на диапазоны
building_param['year_range'] = pd.cut(building_param['Дата постройки'], bins=bins, labels=labels, right=True)


In [723]:
building_param['Этажность объекта'].max()

32

In [739]:
#разбить на категории с помощью get dummies
#static = pd.get_dummies(building_param['year_range'],prefix_sep='_',prefix='year_range',drop_first = True)
static

,year_range_1959-1989,year_range_1990-2000,year_range_2001-2010,year_range_2011-2024
0,False,False,False,True
1,False,False,False,True
2,False,False,False,False
3,False,False,False,True
4,False,False,False,True
...,...,...,...,...
5868,False,False,False,False
5869,False,False,False,False
5870,True,False,False,False
5871,False,False,False,False


In [741]:
#building_param_dummies = pd.concat([building_param,static],axis = 1)
building_param_dummies.columns
#static

Index(['Адрес объекта', 'Тип Объекта', 'Этажность объекта', 'Дата постройки',
       'Общая площадь объекта', 'year_range', 'Number_of_storeys_range',
       'year_range_1959-1989', 'year_range_1990-2000', 'year_range_2001-2010',
       'year_range_2011-2024'],
      dtype='object')

Разбит по группам этажность  (по группам 1-2 этажа, 3-4 этажа, 5-9 этажей,10-12 этажей, 13 и более этажей)

In [724]:
bins = [1, 2, 4, 9, 12, 32]
labels = ['1-2', '3-4', '5-9', '10-12', '> 13']

# Разбивка годов на диапазоны
building_param['Number_of_storeys_range'] = pd.cut(building_param['Этажность объекта'], bins=bins, labels=labels, right=True)


In [727]:
#разбить на категории (кол-во этажей) с помощью get dummies
#static_f = pd.get_dummies(building_param['Number_of_storeys_range'],prefix_sep='_',prefix='Number_of_storeys_range',drop_first = True)

In [754]:
building_param_dummies.set_index('Адрес объекта')

,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,year_range,Number_of_storeys_range,year_range_1959-1989,year_range_1990-2000,year_range_2001-2010,year_range_2011-2024,Number_of_storeys_range_3-4,Number_of_storeys_range_5-9,Number_of_storeys_range_10-12,Number_of_storeys_range_> 13
Адрес объекта,,,,,,,,,,,,,,
"б-р. Баландина, д.11",Многоквартирный дом,18,2011,5394.50,2011-2024,> 13,False,False,False,True,False,False,False,True
"б-р. Баландина, д.2",Многоквартирный дом,16,2012,12731.50,2011-2024,> 13,False,False,False,True,False,False,False,True
"б-р. Баландина, д.2а",Другое строение,3,<NA>,0.00,NaN,3-4,False,False,False,False,True,False,False,False
"б-р. Баландина, д.4",Многоквартирный дом,16,2012,4234.90,2011-2024,> 13,False,False,False,True,False,False,False,True
"б-р. Баландина, д.4 корп.1",Многоквартирный дом,16,2012,4250.10,2011-2024,> 13,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"ул. Юрия Гагарина, д.74 корп.4",Дет.ясли и сады,3,<NA>,0.02,NaN,3-4,False,False,False,False,True,False,False,False
"ул. Юрия Гагарина, д.80",Другое строение,3,<NA>,0.01,NaN,3-4,False,False,False,False,True,False,False,False
"ул. Юрия Гагарина, д.9",Многоквартирный дом,9,1984,5658.50,1959-1989,5-9,True,False,False,False,False,True,False,False


In [762]:
#building_param_dummies = pd.concat([building_param_dummies,static_f],axis = 1)
#building_param_dummies.to_csv('building_param_dummies.csv')

building_param_dummies.columns

Index(['Адрес объекта', 'Тип Объекта', 'Этажность объекта', 'Дата постройки',
       'Общая площадь объекта', 'year_range', 'Number_of_storeys_range',
       'year_range_1959-1989', 'year_range_1990-2000', 'year_range_2001-2010',
       'year_range_2011-2024', 'Number_of_storeys_range_3-4',
       'Number_of_storeys_range_5-9', 'Number_of_storeys_range_10-12',
       'Number_of_storeys_range_> 13'],
      dtype='object')

In [759]:
df_all_temp['Вид энерг-а ГВС'].fillna('not_ГВС-ИТП')
#df_all_temp_building_param_dummies = df_all_temp.merge(building_param_dummies, how = "left", on = "Адрес объекта")

0        not_ГВС-ИТП
1        not_ГВС-ИТП
2        not_ГВС-ИТП
3        not_ГВС-ИТП
4            ГВС-ИТП
            ...     
70404    not_ГВС-ИТП
70405        ГВС-ИТП
70406    not_ГВС-ИТП
70407        ГВС-ИТП
70408    not_ГВС-ИТП
Name: Вид энерг-а ГВС, Length: 70409, dtype: object

In [767]:
#разбить на категории (кол-во этажей) с помощью get dummies
static_g = pd.get_dummies(df_all_temp['Вид энерг-а ГВС'])

In [781]:
static_g

,ГВС-ИТП
0,False
1,False
2,False
3,False
4,True
...,...
70404,False
70405,True
70406,False
70407,True


In [773]:
df_all_temp.head()

,dt,№ ОДПУ,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания,"Текущее потребление, Гкал",avg_temp,days
0,2022-01-01,147256,NaN,"ул. Петергофская, д.2",Другое строение,2022-02-01,24.472,-11.9,31
1,2022-01-01,1740725,NaN,"ул. Кирова, д.103",Многоквартирный дом,2022-02-01,74.652,-11.9,31
2,2022-01-01,247368,NaN,"ул. Трамвайная, д.13 корп.1",Другое строение,2022-02-01,20.781,-11.9,31
3,2022-01-01,98735,NaN,"ул. Достоевского, д.112",Многоквартирный дом,2022-02-01,167.411,-11.9,31
4,2022-01-01,28884,ГВС-ИТП,"ул. Рудольфа Нуреева, д.21",Многоквартирный дом,2022-02-01,282.680,-11.9,31


In [777]:
df_all_temp['Адрес объекта'] = df_all_temp['Адрес объекта'].str.strip()

In [786]:

#df_all_temp_dummies = pd.concat([df_all_temp,static_g],axis = 1)
df_all_temp_dummies_cut = df_all_temp_dummies[['dt','№ ОДПУ','Адрес объекта','Тип объекта',
                                               'Дата текущего показания','Текущее потребление, Гкал','avg_temp','days','ГВС-ИТП']]

In [796]:
df_all_temp_dummies_cut.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70409 entries, 0 to 70408
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   dt                         70409 non-null  object 
 1   № ОДПУ                     70409 non-null  object 
 2   Адрес объекта              70409 non-null  object 
 3   Тип объекта                70409 non-null  object 
 4   Дата текущего показания    70409 non-null  object 
 5   Текущее потребление, Гкал  70409 non-null  float64
 6   avg_temp                   70409 non-null  float64
 7   days                       70409 non-null  int64  
 8   ГВС-ИТП                    70409 non-null  bool   
dtypes: bool(1), float64(2), int64(1), object(5)
memory usage: 4.4+ MB


In [793]:

building_param_dummies_cut = building_param_dummies[['Адрес объекта', 'Тип Объекта','Общая площадь объекта', 'year_range_1959-1989', 'year_range_1990-2000', 'year_range_2001-2010','year_range_2011-2024', 'Number_of_storeys_range_3-4','Number_of_storeys_range_5-9', 'Number_of_storeys_range_10-12','Number_of_storeys_range_> 13']]



In [819]:
#building_param_dummies_cut = building_param_dummies_cut.set_index(['Адрес объекта'])
#df_all_temp_dummies_cut = df_all_temp_dummies_cut.set_index(['Адрес объекта'])
building_param_dummies_cut.index
#building_param_dummies_cut = building_param_dummies_cut.reset_index()
#df_all_temp_dummies_cut = df_all_temp_dummies_cut.reset_index()

RangeIndex(start=0, stop=5693, step=1)

In [820]:
#df_all_temp_dummies_cut.head()

df_all_temp_dummies_cut_build_param = df_all_temp_dummies_cut.merge(building_param_dummies_cut, how = "left", on = "Адрес объекта")
#df_all_temp_dummies_cut_build_param.info()

In [837]:
#len(df_all_temp_dummies_cut_build_param['Адрес объекта'].unique())
#df_all_temp
len(df_all_temp['Адрес объекта'].unique())

4243

In [853]:
#ищу потерянные адреса

len(building_param['Адрес объекта'].unique())

5695

In [854]:
len(building_param_dummies_cut['Адрес объекта'].unique())

5693

In [855]:
df_temp = building_param[building_param['Адрес объекта'].isin(set(building_param_dummies_cut['Адрес объекта']))]
df_temp

,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
5871,респ Башкортостан,Другое строение,0,NaN,0.01
5872,"р-н Уфимский, д Лекаревка",Другое строение,1,NaN,0.01


In [857]:

building_param_dummies_cut[building_param_dummies_cut['Адрес объекта'] == 'р-н Уфимский, д Лекаревка']


,Адрес объекта,Тип Объекта,Общая площадь объекта,year_range_1959-1989,year_range_1990-2000,year_range_2001-2010,year_range_2011-2024,Number_of_storeys_range_3-4,Number_of_storeys_range_5-9,Number_of_storeys_range_10-12,Number_of_storeys_range_> 13
5692,"р-н Уфимский, д Лекаревка",Другое строение,0.01,False,False,False,False,False,False,False,False


In [863]:
df_all_temp_dummies_cut_build_param.to_csv('df_all_temp_dummies_cut_build_param.csv',index=False)
#df_all_temp_dummies_cut_build_param.head()

В таблице df_all_temp (до объединения с build_param имеем 4243 адресов), в таблице build_param имеем большее кол-во адресов
даже после удаления дубликатов (5653).
После объединения таблиц df_all_temp_dummies_cut и building_param_dummies_cut получаем 4243 уникальных адресов.

Попробовать варианты:
- удалить строки с пустыми адресами
- оставить строки с пустыми адресами

